In [1]:
import os
import json

import spotipy.util as util
import requests

In [2]:
with open('settings/webbify_credentials.txt', 'r') as tfile:
    fileContents = tfile.read().split('\n')
    CLIENT_ID = fileContents[0].split(':')[1][1:]
    CLIENT_SECRET = fileContents[1].split(':')[1][1:]

In [93]:
USERNAME = 'nghokui@gmail.com'
TOKEN_CACHE = '.cache-{}'.format(USERNAME)
if os.path.exists(TOKEN_CACHE):
    os.remove(TOKEN_CACHE)

    
AUTH_ARGS = {
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'redirect_uri': 'https://localhost:8888/',
    'scope': 'user-read-recently-played'
}
token = util.prompt_for_user_token(USERNAME, **AUTH_ARGS)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=d5eba668c908462aa3183670fe307302&response_type=code&redirect_uri=https%3A%2F%2Flocalhost%3A8888%2F&scope=user-read-recently-played in your browser


Enter the URL you were redirected to: https://localhost:8888/?code=AQDFxw_67Uv9ph8fmOFGEkwnjueBBFg09f-rvFtjFSiO6Hd99VbN6bOljtMmk0r1lEt1pqxFYuVqGWth_cPCIwsOTAUDLuwfAcOQlICBAAx-c0bMG2uS02FCAPTWpFP26zFnrwTj0Ww4BsM_wAEhTjD6sQSNUalpnBsdiEixLwFhi82R7ZTlKM5ZYxzqZtkQF3NWQLQCJnD-pJiUqGfS78BbSCZj4zO_




In [94]:
URL_BASE = 'https://api.spotify.com'
HEADERS = {'Authorization': 'Bearer {}'.format(token)}

data = requests.get('{}/v1/me'.format(URL_BASE), headers=HEADERS)
data.json()

{'display_name': 'Brian Ng',
 'external_urls': {'spotify': 'https://open.spotify.com/user/nghokui'},
 'followers': {'href': None, 'total': 20},
 'href': 'https://api.spotify.com/v1/users/nghokui',
 'id': 'nghokui',
 'images': [{'height': None,
   'url': 'https://scontent.xx.fbcdn.net/v/t31.0-1/c53.0.320.320a/p320x320/19956194_3497275234192_4181604532212724762_o.jpg?_nc_cat=107&_nc_ohc=SgkbROx5wqgAQk4NE0yipwiHHSuWlRjJfZzZfkQC7riEFblJ-g2UV1_rQ&_nc_ht=scontent.xx&oh=0f7aef9a4414bed54c3966b4b58e7c31&oe=5EA63217',
   'width': None}],
 'type': 'user',
 'uri': 'spotify:user:nghokui'}

In [103]:
# Get a list of playlists
URL_BASE = 'https://api.spotify.com/v1/users/{user_id}/playlists'
HEADERS = {'Authorization': 'Bearer {}'.format(token)}

data = requests.get(URL_BASE.format(user_id='nghokui'), headers=HEADERS, params={'limit': 50})

# Load the data from the request
playlistData = {}
for item in data.json()['items']:
    playlistData[item['name']] = {
        'URI': item['uri'],
        'Private': item['public'],
        'TrackList': item['tracks']['href'],
        'NumTracks': item['tracks']['total']
    }

In [105]:
# Get the tracklist
userID = 'nghokui'
playlistName = 'things'
plist = playlistData[playlistName]

queryURL = plist['TrackList']
continueRun = True

playlistTracks = []
while queryURL:
    queryData = requests.get(queryURL, headers=HEADERS)
    
    if str(queryData) == '<Response [200]>':
        data = queryData.json()
        for item in data['items']:
            trackItem = {}
            
            addedTime = item['added_at']
            
            trackInfo = item['track']
            albumInfo = trackInfo['album']

            trackItem = {
                'id': trackInfo['id'],
                'name': trackInfo['name'],
                'popularity': trackInfo['popularity'],
                'duration': trackInfo['duration_ms'],
                'album': {
                    'name': albumInfo['name'],
                    'type': albumInfo['album_type'],
                    'release': albumInfo['release_date'],
                    'releasePrecision': albumInfo['release_date_precision'],
                    'id': albumInfo['uri']
                }
            }
            
            artistData = []
            artistInfo = trackInfo['artists']
            for artist in artistInfo:
                artistData.append((
                    artist['name'],
                    artist['type'],
                    artist['uri']
                ))
            
            trackItem['artist'] = artistData
            
            # Add track item
            playlistTracks.append(trackItem)
        queryURL = data['next']
    else:
        queryURL = None

playlist = {
    'meta': plist,
    'data': None
}
playlist['data'] = playlistTracks
with open('data/{}/playlists/{}_Tracklist.json'.format(userID, playlistName), 'w') as jfile:
    json.dump(playlist, jfile)

In [106]:
len(playlistTracks)

667

General Info:
1. Track url: https://api.spotify.com/v1/tracks/<track ID>

In [5]:
# Get recently played
recentlyPlayedHeaders = HEADERS.copy()
recentlyPlayedHeaders['limit'] = '50'

data = requests.get('{}/v1/me/player/recently-played'.format(URL_BASE), headers=HEADERS, payload=)
data.json()

SyntaxError: invalid syntax (<ipython-input-5-7c13bd04474d>, line 5)

In [20]:
for i,item in enumerate(data.json()['items']):
    track = item['track']
    
    
    tn = track['name']
    tan = track['artists'][0]['name']
    tal = track['album']['name']
    tau = track['uri']
    
    print(i, '||', tn, '||', tan, '||', tal, '||', tau)

data.json()['next']

0 || ocean eyes - Recorded Live at Jungle City Studios || Alicia Keys || Spotify Singles || spotify:track:7rLokcIMP9p8fl0iROdVfC
1 || ocean eyes - Recorded Live at Jungle City Studios || Alicia Keys || Spotify Singles || spotify:track:7rLokcIMP9p8fl0iROdVfC
2 || ocean eyes - Recorded Live at Jungle City Studios || Alicia Keys || Spotify Singles || spotify:track:7rLokcIMP9p8fl0iROdVfC
3 || ocean eyes - Recorded Live at Jungle City Studios || Alicia Keys || Spotify Singles || spotify:track:7rLokcIMP9p8fl0iROdVfC
4 || ocean eyes - Recorded Live at Jungle City Studios || Alicia Keys || Spotify Singles || spotify:track:7rLokcIMP9p8fl0iROdVfC
5 || ocean eyes - Recorded Live at Jungle City Studios || Alicia Keys || Spotify Singles || spotify:track:7rLokcIMP9p8fl0iROdVfC
6 || Family || The Chainsmokers || World War Joy || spotify:track:3TKpJrY9q49Mj1JOsM9zGL
7 || ocean eyes - Recorded Live at Jungle City Studios || Alicia Keys || Spotify Singles || spotify:track:7rLokcIMP9p8fl0iROdVfC
8 || Fin

'https://api.spotify.com/v1/me/player/recently-played?before=1575610712335'

In [21]:
data = requests.get(data.json()['next'], headers=recentlyPlayedHeaders)

AttributeError: 'Response' object has no attribute 'response'

In [22]:
for i,item in enumerate(data.json()['items']):
    track = item['track']
    
    
    tn = track['name']
    tan = track['artists'][0]['name']
    tal = track['album']['name']
    tau = track['uri']
    
    print(i, '||', tn, '||', tan, '||', tal, '||', tau)

data.json()['next']

0 || Dragonstone || Ramin Djawadi || Game Of Thrones: Season 7 (Music from the HBO Series) || spotify:track:65KG1BEtR85y2cvIaukuaO
1 || Journey (feat. Kronos Quartet) || Michael Salvatori || Destiny 2 (Original Soundtrack) || spotify:track:5P68xT9dqkleXVVtdIJPXo
2 || Dr. Ford || Ramin Djawadi || Westworld: Season 1 (Music from the HBO Series) || spotify:track:1pm8TqG9dNIcMGTQNzjYh9
3 || Another Way || Clinton Shorter || The Expanse Season 3 (Original Television Soundtrack) || spotify:track:6Id2J66eXJCtiXLazbsh3K
4 || The Expanse || Clinton Shorter || The Expanse (Original Television Soundtrack) || spotify:track:7FrCiWJbNiSbDNih6IzyAA
5 || Hammerlock || Clinton Shorter || The Expanse Season 3 (Original Television Soundtrack) || spotify:track:1K9AzZIL0GNwxpT16bsOQp
6 || Runaway || Ramin Djawadi || Westworld: Season 2 (Music From the HBO Series) || spotify:track:4NBYp73qsxFh9yUUnMy6jz
7 || His Dark Materials (Main Title Theme) || Baltic House Orchestra || His Dark Materials (Main Title Th

'https://api.spotify.com/v1/me/player/recently-played?before=1575605877508'